In [4]:
import os
import importlib
from utility import Pi_util
from natsort import natsorted
import numpy as np
import pickle
import sys
import pandas as pd

path_to_lib: str = "/Users/zacharykelly/Documents/MATLAB/projects/combiExperiments/code/lightLogger/miniSpect"
sys.path.append(path_to_lib)
import MS_util
import importlib

In [5]:
experiment_path: str = "/Volumes/EXTERNAL1/actuallyFixedBuffer_0.1hz_0NDF"
use_mean_frame: bool = True

In [6]:
Pi_util.parse_chunks_pkl(experiment_path, use_mean_frame)

Loading chunk: 1/60
Loading chunk: 2/60
Loading chunk: 3/60
Loading chunk: 4/60
Loading chunk: 5/60
Loading chunk: 6/60
Loading chunk: 7/60
Loading chunk: 8/60
Loading chunk: 9/60
Loading chunk: 10/60
Loading chunk: 11/60
Loading chunk: 12/60
Loading chunk: 13/60
Loading chunk: 14/60
Loading chunk: 15/60
Loading chunk: 16/60
Loading chunk: 17/60
Loading chunk: 18/60
Loading chunk: 19/60
Loading chunk: 20/60
Loading chunk: 21/60
Loading chunk: 22/60
Loading chunk: 23/60
Loading chunk: 24/60
Loading chunk: 25/60
Loading chunk: 26/60
Loading chunk: 27/60
Loading chunk: 28/60
Loading chunk: 29/60
Loading chunk: 30/60
Loading chunk: 31/60
Loading chunk: 32/60
Loading chunk: 33/60
Loading chunk: 34/60
Loading chunk: 35/60
Loading chunk: 36/60
Loading chunk: 37/60
Loading chunk: 38/60
Loading chunk: 39/60
Loading chunk: 40/60
Loading chunk: 41/60
Loading chunk: 42/60
Loading chunk: 43/60
Loading chunk: 44/60
Loading chunk: 45/60
Loading chunk: 46/60
Loading chunk: 47/60
Loading chunk: 48/60
L

[{'W': {'frame_buffer': array([78.2375    , 78.31583333, 78.37416667, ..., 78.1375    ,
          78.24166667, 78.3175    ]),
   'settings_buffer': array([[1.00e+00, 1.00e+02],
          [1.00e+00, 1.00e+02],
          [1.00e+00, 1.00e+02],
          ...,
          [2.00e+00, 4.84e+03],
          [2.00e+00, 4.84e+03],
          [2.00e+00, 4.84e+03]]),
   'num_frames_captured': 2006.0,
   'FPS': 200.5441921279307},
  'M': {'A':       0    1   2    3    4    5    6   7     8   9
   0  4206  124  70  123  137  173  118  76  7914   8
   1  5019  148  84  148  160  202  137  88  9236  11
   2  5233  155  88  155  163  206  140  89  9411  11
   3  4776  141  80  141  145  182  124  80  8315  10
   4  3839  114  63  112  113  141   98  62  6457   8
   5  2767   82  45   80   80  100   69  43  4517   6
   6  2003   60  32   58   59   74   51  31  3301   4
   7  1866   56  30   54   58   73   51  31  3282   4
   8  2387   71  39   69   79   99   68  42  4465   6
   9  3379  100  56   98  112  1